In [1]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
Click <a href="javascript:code_toggle()">here</a> to see the code.''')

In [2]:
import ipywidgets as ipw
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
plt.style.use('seaborn-poster')

from tabulate import tabulate

def matProps(mat):
    if mat == 'CuNi10':
        E = 140e9
        nu = 0.35
        rho = 8940
    elif mat == 'Cu':
        E = 110e9
        nu = 0.35
        rho = 7940
    elif mat == 'Al':
        E = 69e9
        nu = 0.35
        rho = 2700
    return E, nu, rho

def tubeFinParams(config):
    if config == 'EK2334':
        dx = 23e-3
        dy = 34e-3
        R = (15e-3)/2
        tFin = 0.18e-3
        tTube = 0.80e-3
    if config == 'EK2334 (VaCN)':
        dx = 23e-3
        dy = 34e-3
        R = (15e-3)/2
        tFin = 0.14e-3
        tTube = 0.80e-3
    if config == 'CK1619':
        dx = 16e-3
        dy = 19e-3
        R = (10.55e-3)/2
        tFin = 0.12e-3
        tTube = 0.60e-3
    if config == 'UCK1212':
        dx = 12e-3
        dy = 12e-3
        R = (7.5e-3)/2
        tFin = 0.12e-3
        tTube = 0.50e-3
    return dx, dy, R, tFin, tTube

def paras(tubeMat, finMat, config):
    dx, dy, R, tFin, tTube = tubeFinParams(config)
    ETube, nuTube, rhoTube = matProps(tubeMat)
    EFin, nuFin, rhoFin = matProps(finMat)
    return dx, dy, nuTube, R, EFin, ETube, rhoFin, rhoTube, tFin, tTube

def getBetaVals():
    from scipy.optimize import brentq
    x = np.linspace(2, 20, 100)
    f = lambda x: -1+np.cos(x)*np.cosh(x)
    cross = f(x[:-1])*f(x[1:])
    idx = np.where(cross<0)[0]
    rts = []
    for i in idx:
        rts.append(brentq(f, x[i-1], x[i+1]))
    return rts

def getFreqs(EFin, ETube, nuTube, dx, dy, nRow, nCol, R, rhoTube, rhoFin, tTube, tFin, L, Nf, finFlexCorr):
    GTube = ETube/(2*(1 + nuTube))
    betaVals = getBetaVals()
    x1 = dx/2 - dy**2/(8*dx)
    x2 = dx/2 + dy**2/(8*dx)
    Rsq = 1/48*nCol*nRow*(4*dx**2*(-1 + nCol**2) +dy**2*(-1 + 4*nRow**2))
    aUnit = dy*(x1 + x2) - np.pi*R**2
    JUnit = (3*dy**3*x1 + 4*dy*x1**3 + dy**3*x2 + 4*dy*x1**2*x2 + 4*dy*x1*x2**2 + 4*dy*x2**3)/24 - np.pi*R**4/2
    aTube = np.pi*(R**2 - (R - tTube)**2)
    ITube = np.pi/4*(R**4 - (R - tTube)**4)
    JTube = np.pi/2*(R**4 - (R - tTube)**4)
    Luc = L/Nf
    NTube = nRow*nCol
    JTotal = JUnit*NTube + aUnit * Rsq
    
    fTwist = [];
    fBend = [];
    [A, B, C] = getConstRigid(L)
    fT = (1/(2*np.pi))*np.sqrt((ETube*ITube*Rsq*B + NTube*GTube*JTube*C)/(A*(rhoTube*aTube*Rsq + JTotal*rhoFin*tFin/Luc)))
    fTwist.append(fT)

    for i in betaVals:
        [A, B, C] = getConst(i, L)
        fT = (1/(2*np.pi))*np.sqrt((ETube*ITube*Rsq*B + NTube*GTube*JTube*C)/(A*(rhoTube*aTube*Rsq + JTotal*rhoFin*tFin/Luc)))
        fTwist.append(fT)
        
        b = dx
        IFin = 1/12*dy*tFin**3
        muTube = aTube*rhoTube
        muFin = rhoFin*aUnit*tFin/b
        
        fB = np.sqrt((ETube*ITube*B+finFlexCorr*12*EFin*IFin/(b*Luc)*C)/(muTube*A+muFin*b/Luc*A))/(2*np.pi)
        fBend.append(fB)
    
    return fBend, fTwist

    
def getConstRigid(L):
    A = (1/12)*L
    B = 0/L**3
    C = 1/L
    
    return A, B, C
    

def getConst(beta, L):
    alpha = (np.cos(beta)-np.cosh(beta))/(np.sin(beta)-np.sinh(beta))
    
    A = (4*beta+2*alpha*np.cos(2*beta)-2*alpha*np.cosh(2*beta)+4*(1+alpha**2)*np.cosh(beta)*np.sin(beta)+np.sin(2*beta)-alpha**2*np.sin(2*beta)+4*np.cos(beta)*np.sinh(beta)-4*alpha**2*np.cos(beta)*np.sinh(beta)-8*alpha*np.sin(beta)*np.sinh(beta)+np.sinh(2*beta)+alpha**2*np.sinh(2*beta))/(4*beta)
    B = 1/4*beta**3*(4*beta+2*alpha*np.cos(2*beta)-2*alpha*np.cosh(2*beta)-4*(1+alpha**2)*np.cosh(beta)*np.sin(beta)+np.sin(2*beta)-alpha**2*np.sin(2*beta)-4*np.cos(beta)*np.sinh(beta)+4*alpha**2*np.cos(beta)*np.sinh(beta)+8*alpha*np.sin(beta)*np.sinh(beta)+np.sinh(2*beta)+alpha**2*np.sinh(2*beta))
    C = 1/4*beta*(12*alpha+4*alpha**2*beta-2*alpha*np.cos(2*beta)-2*alpha*np.cosh(2*beta)+4*np.cosh(beta)*(-2*alpha*np.cos(beta)+(-1+alpha**2)*np.sin(beta))+(-1+alpha**2)*np.sin(2*beta)+4*(1+alpha**2)*np.cos(beta)*np.sinh(beta)+np.sinh(2*beta)+alpha**2*np.sinh(2*beta))
    
    return A*L, B/L**3, C/L

***
### Bedning and Twisting Mode Frequencies
<b><font color='#1f77b4'>Bending Mode</font> 

<font color='#ff7f0e'> Twisting Mode</font></b>

In [3]:
def plotter(nRow, nCol, Nf, L, maxFreq, tubeMat, finMat, config, finFlexCorr):
    nRow = int(nRow)
    nCol = int(nCol)
    Nf = int(Nf)
    L = float(L)
    dx, dy, nuTube, R, EFin, ETube, rhoFin, rhoTube, tFin, tTube = paras(tubeMat, finMat, config)
    fBend, fTwist = getFreqs(EFin, ETube, nuTube, dx, dy, nRow, nCol, R, rhoTube, rhoFin, tTube, tFin, L, Nf, finFlexCorr)
    f = np.append(fBend, fTwist)
    idx = np.argsort(f)
    bt = np.array(['C0']*len(fBend) + ['C1']*len(fTwist))
    bt = bt[idx]
    f = f[idx]
#     plt.bar(range(len(f)), f, color = bt)
#     plt.ylabel('Freq (Hz)')
#     plt.ylim([0, 1000])
#     plt.xticks([])
#     plt.plot(plt.xlim(), [maxFreq, maxFreq], color = 'C3' )
#     plt.show()
    freqRounded = ['Freq (Hz): '] + list(int(round(num)) for num in f)
    modeType = np.array(['Bend']*len(fBend) + ['Twist']*len(fTwist))
    modeType = ['ModeType: '] + list(modeType[idx])
    print(tabulate([freqRounded, modeType]))
    
interactive_plot = ipw.interactive(plotter,
                               nRow = '13',
                               nCol='4',
                               Nf = '262',
                               L = '0.515',
                               maxFreq = (0, 300, 10),
                               tubeMat = ['CuNi10', 'Cu', 'Al'],
                               finMat = ['Cu', 'CuNi10', 'Al'],
                               config = ['EK2334', 'EK2334 (VaCN)', 'CK1619', 'UCK1212'],
                               finFlexCorr= (0, 10, 0.2),
                               )
display(interactive_plot)

interactive(children=(Text(value='13', description='nRow'), Text(value='4', description='nCol'), Text(value='2…